In [31]:
import numpy as np

test_file_path = "../data/day15/test.txt"
file_path = "../data/day15/input.txt"

def load_data(file_path):
    with open(file_path) as file:
        size = len(file.readline().strip())
    return np.genfromtxt(file_path, delimiter=np.ones(size, dtype='int8'), dtype='int8')

In [13]:
def shortest_unvisited(input_dictionary):
  min_distance = np.iinfo(np.int32).max
  min_position = (0,0)
  for position, distance in input_dictionary.items():
    if distance < min_distance:
      min_position = position
      min_distance = distance
  return (min_position)

In [21]:
def neighbours(array, position):
    index_inc = np.array([[0,1],[0,-1],[1,0],[-1,0]])
    positions = position + index_inc
    positions = positions[((positions >= (0,0)) & (positions < array.shape)).all(axis=1)]
    return tuple(map(tuple, positions))

In [52]:
def shortest_path(start, end, weights):
  unvisited = {(x, y) : np.iinfo(np.int32).max
                 for x in range(weights.shape[0]) for y in range(weights.shape[1])}

  distances = {(x, y) : np.iinfo(np.int32).max
                 for x in range(weights.shape[0]) for y in range(weights.shape[1])}
  current_position = start
  unvisited[start] = 0
  while current_position != end and len(unvisited) > 0:
    current_distance = unvisited[current_position]
    distances[current_position] = current_distance
    del unvisited[current_position]
    for position in neighbours(weights, current_position):
      if position in unvisited:
        new_distance = weights[position] + current_distance
        old_distance = unvisited[position]
        if new_distance < old_distance:
          unvisited[position] = new_distance
    current_position = shortest_unvisited(unvisited)
  if current_position == end:
    return unvisited[end]
  else:
    return None

weights = load_data(file_path)
shortest_path((0,0),(99,99), weights)

698